# 1. Introduction

## 1.1 Imports
Import libraries here.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from kuberspatiotemporal import CompoundModel, Feature, SpatialModel, KuberModel
from kuberspatiotemporal.tools import make_ellipses

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

# 2. Load Data

In [ ]:
data = pd.read_json('data/spatial_data_lisboa.json')

In [ ]:
data.head(2)

In [ ]:
#array([[38.7002492, -9.149562 ],
#       [38.7365033, -9.1234308]])
data = data[['latitude', 'longitude', 'timestamp']]
data = data[((data['latitude']>38.7002492) & (data['latitude']<38.7365033)) & ((data['longitude']> -9.149562) & (data['longitude']<-9.1234308))]
data

In [ ]:
# Filter office and home locations
data = data[['latitude', 'longitude', 'timestamp']]
data = data[(
    (((data['latitude']>38.710104) & (data['latitude']<38.710648)) & ((data['longitude']>-9.139794) & (data['longitude']<-9.1389)))
 | (((data['latitude']>38.725828) & (data['latitude']<38.726512)) & ((data['longitude']>-9.133944) & (data['longitude']<-9.133169))))]

data



In [ ]:
spatial_data = data[['latitude', 'longitude' ]] 

In [ ]:
grouped_locations = data.groupby(['latitude', 'longitude']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Rectangle

m = Map(center=(np.mean(data.latitude), np.mean(data.longitude)), zoom=10)

for lat, lon, count in zip(grouped_locations.latitude.values, grouped_locations.longitude.values, grouped_locations.Count.values):
    circle_marker = CircleMarker()
    circle_marker.location = (lat, lon)
    circle_marker.radius = int((count-1)*(25-3)/(np.max(grouped_locations.Count.values)-np.min(grouped_locations.Count.values))+3)
    circle_marker.color = "blue"
    circle_marker.fill_color = "blue"

    m.add_layer(circle_marker)

m

In [ ]:
limits = np.array([[data.latitude.min(), data.longitude.min()],[data.latitude.max(), data.longitude.max()]])
limits = limits + np.array([[-0.01], [0.01]])

In [ ]:
limits

In [ ]:
m.add_layer(Rectangle(bounds=limits.tolist()))

# 3. Learn Spatial Model - 2D

**scaling_parameter**

The scaling parameter of the dirichlet proccess.

**min_eigval**

Important value. Minimum extend a cluster/component is allowed to have in one of its main directions. Prevents degenerated components. Read the documentation for details, defaults to `1e-2`.

In [ ]:
model = SpatialModel(n_dim=2, min_eigval=0.000000001, nonparametric=True, n_iterations=200, limits=limits,
                    scaling_parameter=100, loa=True, decay=5)

In [ ]:
idx = np.argsort(model._weights)

In [ ]:
model.fit(data[['latitude', 'longitude']].values)

In [ ]:
idx = np.argsort(model._weights)
idx

In [ ]:
# weights 
# model._weights
# covs
# model._SpatialModel__covs
# means
# model._SpatialModel__means

In [ ]:
model._weights

In [ ]:
model._SpatialModel__means

In [ ]:
model.box = 0.05

In [ ]:
#home
model.score(np.array([[38.7263483,-9.135931]]))

In [ ]:
#office
model.score(np.array([[38.7104174,-9.1417113]]))

## 3.1 Prediction grid

In [ ]:
from typing import Union
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from scipy.stats import mvn
from scipy.stats._multivariate import _squeeze_output

def is_broadcastable(a: Union[float, np.ndarray], b: Union[float, np.ndarray]) -> bool:
    r"""Checks whether two objects can be
    `broadcast to each other <https://stackoverflow.com/a/47244284>`_
    """
    return all((m == n) or (m == 1) or (n == 1) for m, n in zip(a.shape[::-1], b.shape[::-1]))

def boxed_cumulative(gmm: Union[GaussianMixture, BayesianGaussianMixture],
                     centers: np.ndarray, width: Union[float, np.ndarray],
                     maxpts: float = None, abseps=1e-5, releps=1e-5) -> Union[float, np.ndarray]:
    r"""Compute the *boxed* cumulative density of a Gaussian Mixture Model function given the centers and
    widths of one or more intervals.

    Parameters
    ----------
    centers : ndarray
        The centers of the boxes to compute the cumulative function for. Row matrix
        of points. Number of columns must match the dimension of the mean parameter.
    width : float or ndarray
        The width of the box. Must be able to be broadcast to the first parameter.
    mean : ndarray
        See :data:`scipy.stats.multivariate_normal`
    cov : ndarray
        See :data:`scipy.stats.multivariate_normal`
    maxpts : float
        See :data:`scipy.stats.multivariate_normal`
    abseps : float
        See :data:`scipy.stats.multivariate_normal`
    releps : float
        See :data:`scipy.stats.multivariate_normal`
    """
    # XXX SU->SU: Docstring incomplete / requires checking
    return np.sum(
        [
            weight *
            boxed_cdf(centers, width, mean, sigma, maxpts, abseps, releps)
            for sigma, mean, weight in zip(gmm.covariances_, gmm.means_, gmm.weights_)
        ]
    )


def boxed_cdf(
        centers: np.ndarray, width: Union[float, np.array],
        mean: np.ndarray, cov: np.ndarray,
        maxpts: float = None, abseps=1e-5, releps=1e-5) -> Union[float, np.ndarray]:
    """
    Compute the *boxed* cumulative density function given the centers and
    widths of one or more intervals.

    Parameters
    ----------
    centers : ndarray
        The centers of the boxes to compute the cumulative function for. Row matrix
        of points. Number of columns must match the dimension of the mean parameter.
    width : float or ndarray
        The width of the box. Must be able to be broadcast to the first parameter.
    mean : ndarray
        See :data:`scipy.stats.multivariate_normal`
    cov : ndarray
        See :data:`scipy.stats.multivariate_normal`
    maxpts : float
        See :data:`scipy.stats.multivariate_normal`
    abseps : float
        See :data:`scipy.stats.multivariate_normal`
    releps : float
        See :data:`scipy.stats.multivariate_normal`

    Returns
    -------
    cdf : ndarray or scalar
        Cumulative distribution function evaluated at `x`


    Notes
    -----
    The code is inspired by scipy.stats._multivariate.

    """
    # XXX SU->SU: improve doc string. Missing several parameters

    dim = mean.size

    # Increase dimensionality to 2D if necessary
    centers = np.atleast_2d(centers)
    width = np.atleast_2d(width)

    # check if dimensions are compatible
    assert centers.shape[1] == dim
    assert is_broadcastable(centers, width)

    # We construct a matrix with the intervals defined in the rows
    # the first half of the components are the lower bound,
    # the second half the upper bound.

    lower_upper = np.hstack((centers-width/2., centers+width/2.))

    if not maxpts:
        maxpts = 1000000 * dim

    # mvnun expects 1-d arguments, so process points sequentially
    # We apply the computation along the last axis, so that we
    # process the rows in parallel.

    out = np.apply_along_axis(
        lambda stacked: mvn.mvnun(  # Computes the boxed CDF (fortran wrapper)
            stacked[0:dim],         # First columns represent the lower bound
            stacked[dim:],          # Last columns represent the upper bound
            mean, cov,              # The parameters of the normal distribution
            maxpts, abseps, releps  # Parameters of the algorithm
        )[0], -1, lower_upper
    )
    if np.isnan(out):
        out = np.array([0])

    return _squeeze_output(out)

In [ ]:
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
from ipyleaflet import (
    Rectangle
)
def pcolorOnMap(xx: np.ndarray,
                yy: np.ndarray,
                zz: np.ndarray,
                samples: np.ndarray = None):
    r"""
    Helper plot function for displaying a grid of rectangles on a map.

    Parameters
    ----------
    xx: shape (m, n)
        Array created by :func:`numpy.meshgrid`

    yy: shape (m, n)
        Array created by :func:`numpy.meshgrid`

    zz: shape (m, n)
        Function values of a function :math:`f(xx,yy)`.
        Values of the range 0 to 1.

    samples: shape(2,k)
        Samples (:math:`k`) to be displayed on the map.

    Notes
    -----
    Only implemented if  `ipyleaflet <https://github.com/jupyter-widgets/ipyleaflet>`_ is installed.

    """
    width = (0.5*(xx[0, 1]-xx[0, 0]), 0.5*(yy[1, 0]-yy[0, 0]))
    center = (0.5*(xx[0, -1]+xx[0, 0]), 0.5*(yy[-1, 0]+yy[0, 0]))

    boxes = np.array([xx.flatten(), yy.flatten()]).T

    cmap = plt.cm.get_cmap("viridis")
    world = Map(center=center, zoom=13)

    if samples is not None:
        for loc in samples:
            circle_marker = CircleMarker()
            circle_marker.location = loc.tolist()
            circle_marker.radius = 2
            circle_marker.color = "red"
            circle_marker.fill_color = "red"
            world.add_layer(circle_marker)

    for box, value in zip(boxes, zz.flatten()):
        message = HTML()
        message.value = f"{value*100:.1f}%"
        color = cmap(value/np.max(zz))
        color_hex = to_hex(color, keep_alpha=False)
        rectangle = Rectangle(bounds=((box[0] - width[0], box[1] - width[1]),
                                      (box[0] + width[0], box[1] + width[1])),
                              weight=0, fill_color=color_hex, stroke=False, fill_opacity=0.5)
        world.add_layer(rectangle)
        rectangle.popup = message

    return world

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, HTML

steps=20
# Acquire some sample for getting the boundaries
samples = model.rvs(1000)

# pick min and max values but widen interval by 25%
max_ = np.max(samples, axis=0)
min_ = np.min(samples, axis=0)
width = (max_-min_)

max_ += width * 0.25
min_ -= width * 0.25
width = (max_-min_)

# Compute the probability for each *combination* of x/y values

# we compare sklearn.mixtures.GaussianMixture.score_samples (log if the PDF)
# with our B-CDF approach

xx, yy = np.meshgrid(
    np.linspace(min_[0], max_[0], steps),
    np.linspace(min_[1], max_[1], steps),
)
scores = np.zeros(xx.shape)
probabilities = np.zeros(xx.shape)
probabilities_ = np.zeros(xx.shape)


for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        center = [[xx[i, j], yy[i, j]]]
        probabilities[i, j] =  np.sum(
        [
            weight *
            boxed_cdf(center, width/steps, mean, sigma, maxpts=None, abseps=1e-5, releps=1e-5)
            for sigma, mean, weight in zip(model._SpatialModel__covs, model._SpatialModel__means, model._weights)
        ]
    )


display(pcolorOnMap(xx,yy,probabilities, model._SpatialModel__means))


In [ ]:
from kuberspatiotemporal.tools import make_ellipses

In [ ]:
f, ((ax1, ax2)) = plt.subplots(1,2, figsize=(18, 10))
ax1.scatter(data.latitude, data.longitude, marker='o',
            s=25, edgecolor='k')
ax1.set_title('data')

make_ellipses(model, ax2)
ax2.scatter(data.latitude, data.longitude,
            s=25, edgecolor='k')
ax2.set_title('ellipses')

In [ ]:
model

# 4. Learn Spatiotemporal Model 

In [ ]:
data.timestamp

In [ ]:
# will be added to the spatial model and interpreted as continuous
data['time'] = [ts.hour + ts.minute/60 + ts.second/3600 for ts in data.timestamp]
# weekday will be part of another model
data['weekday'] = [ts.dayofweek for ts in data.timestamp]

In [ ]:
limits_ = [np.min(data[['latitude', 'longitude', 'time']].values, axis=0),np.max(data[['latitude', 'longitude', 'time']].values, axis=0)]

In [ ]:
kst = CompoundModel(
    n_dim=4,
    n_iterations=200,
    scaling_parameter=1.1,
    nonparametric=True,
    online_learning=False,
    score_threshold=0.85,
    loa=True,
    features=[
        Feature(SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits_), [0, 1, 2]),
        Feature(KuberModel(n_symbols=7), [3])
    ],
)

In [ ]:
pipeline = make_pipeline(
    make_column_transformer(
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "latitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "longitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "time"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "weekday"),
    ),
    kst,
)

In [ ]:
pipeline.fit(data[['latitude', 'longitude', 'time', 'weekday']])

In [ ]:
pipeline.predict(data[['latitude', 'longitude', 'time', 'weekday']].loc[0:2])

In [ ]:
data[['latitude', 'longitude', 'time', 'weekday']].iloc[0]

In [ ]:
kst

In [ ]:
data[['latitude', 'longitude', 'time', 'weekday']].iloc[20:22]

In [ ]:
kst.features[0].model.box = np.array([0.5,0.5,1])

In [ ]:
pipeline.score(data[['latitude', 'longitude', 'time', 'weekday']])

In [ ]:
data_ = data[['latitude', 'longitude', 'time', 'weekday']].iloc[21:22].values

In [ ]:
1 - kst.features[1].model.expect(data_[:, kst.features[1].columns]) * kst._weights[np.newaxis, :]

In [ ]:
np.prod(1 - kst.features[1].model.expect(data_[:, kst.features[1].columns]) * kst._weights[np.newaxis, :])

In [ ]:
1 - kst.features[0].model.expect(data_[:, kst.features[0].columns]) * kst._weights[np.newaxis, :]

In [ ]:
np.sum(kst.features[1].model.expect(data_[:, kst.features[1].columns]) * kst._weights[np.newaxis, :])

In [ ]:
np.sum(kst.features[0].model.expect(data_[:, kst.features[0].columns]) * kst._weights[np.newaxis, :])